# Push/Pull to AWS
* Training DF files
* then push other data

In [1]:
import pandas as pd
import boto3
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from tqdm import tqdm_notebook
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")
#set path directory

#load access key
#HOME = os.path.expanduser('~')
HOME = os.getcwd()
# HOME = os.chdir('../..')
# HOME = os.getcwd()
KEYPATH = "AWSaccessKeys.csv"
ACCESS = pd.read_csv(f"{HOME}/{KEYPATH}")

#start session
SESSION = boto3.Session(
    aws_access_key_id=ACCESS['Access key ID'][0],
    aws_secret_access_key=ACCESS['Secret access key'][0],
)
S3 = SESSION.resource('s3')
#AWS BUCKET information
BUCKET_NAME = 'national-snow-model'
BUCKET = S3.Bucket(BUCKET_NAME)


FileNotFoundError: [Errno 2] No such file or directory: '/uufs/chpc.utah.edu/common/home/civil-group1/Johnson/SWEMLv2.0/Dataprocessing/AWSaccessKeys.csv'

In [2]:
HOME

'/uufs/chpc.utah.edu/common/home/civil-group1/Johnson'

## Training DF regional files

In [ ]:
#Upload nearest-snotel dictionaries
regionlist = ['SouthernRockies', 'Southwest', 'Northwest']
output_res = 300

for region in regionlist:
    print(region)
    head_folder = f"SWEMLv2.0/data/TrainingDFs/{region}/300M_Resolution"
    head_folder_dir = f"{HOME}/{head_folder}"

    #Get files only
    files = [f for f in listdir(head_folder_dir) if isfile(join(head_folder_dir, f))]
    # for file in tqdm_notebook(files):
    #     S3.meta.client.upload_file(Filename= f"{head_folder_dir}{file}", Bucket=BUCKET_NAME, Key=f"{head_folder}{file}")

    #get list of directories to download
    dirs = [ f.path for f in os.scandir(head_folder_dir) if f.is_dir() ]
    for dir in dirs:
        print(f"Folder name: {dir.split('300M_Resolution/')[-1]}")
        awsfolderpath = f"SWEMLv2.0{dir.split('SWEMLv2.0')[-1]}"
        headfolderpath = f"{HOME}/{awsfolderpath}"
        files = [f for f in listdir(headfolderpath) if isfile(join(headfolderpath, f))]
        # for file in tqdm_notebook(files):
        #      S3.meta.client.upload_file(Filename= f"{headfolderpath}/{file}", Bucket=BUCKET_NAME, Key=f"{awsfolderpath}/{file}")
        #     # print(f"{headfolderpath}/{file}")

        # # #Get subfolders
        subdirs = [ f.path for f in os.scandir(headfolderpath) if f.is_dir() ]
        for dir in subdirs:
            print(f"Folder name: {dir.split('300M_Resolution/')[-1]}")
            awsfolderpath2 = f"SWEMLv2.0{dir.split('SWEMLv2.0')[-1]}"
            headfolderpath2 = f"{HOME}/{awsfolderpath2}"
            files = [f for f in listdir(headfolderpath2) if isfile(join(headfolderpath2, f))]
            for file in tqdm_notebook(files):
                S3.meta.client.upload_file(Filename= f"{headfolderpath2}/{file}", Bucket=BUCKET_NAME, Key=f"{awsfolderpath2}/{file}")



        




## Get files

In [4]:
#Upload nearest-snotel dictionaries
regionlist = ['Northwest','SouthernRockies', 'Southwest' ]
output_res = 500

#Get data
for region in regionlist:
    print(region)
    path = f"SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution"

    #Make directory if it does not exist
    if not os.path.exists(f"{HOME}/{path}"):
        print("Path not present, making")
        os.makedirs(f"{HOME}/{path}", exist_ok=True)
    
    #Identify the potential files to download
    files = [objects.key for objects in BUCKET.objects.filter(Prefix=path)]
    #print(files)
    
    for file in tqdm_notebook(files):
        #Make directory if it does not exist
        if not os.path.exists(f"{HOME}/{path}"):
            print("Path not present, making")
        os.makedirs( os.path.dirname(f"{HOME}/{file}"), exist_ok=True)
       
        #check to see if the file is there
        if os.path.exists(f"{HOME}/{file}") == False:
            print('Downloading file')
            S3.Bucket(BUCKET_NAME).download_file(file, f"{HOME}/{file}")

        else:
            print('File already present...')


Northwest
['SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/ASO_meta.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/GeoObsDFs/GeoObsdfs_20160208.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/GeoObsDFs/GeoObsdfs_20160329.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/Northwest_metadata.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/Obsdf/20160208_ObsDF.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/Obsdf/20160329_ObsDF.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/PrecipVIIRSGeoObsDFs/20_fSCA_Thresh/Precip_VIIRS_GeoObsDF_20160208.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/PrecipVIIRSGeoObsDFs/20_fSCA_Thresh/Precip_VIIRS_GeoObsDF_20160329.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/Seasonality_PrecipVIIRSGeoObsDFs/20_fSCA_Thresh/Season_Precip_VIIRS_GeoObsDF_20160208.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/Seasonality

  0%|          | 0/17 [00:00<?, ?it/s]

File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
Northwest
['SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/ASO_meta.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/GeoObsDFs/GeoObsdfs_20160208.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/GeoObsDFs/GeoObsdfs_20160329.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/Northwest_metadata.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/Obsdf/20160208_ObsDF.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/Obsdf/20160329_ObsDF.parquet', 'SWEMLv2.0/data/TrainingDFs/Northwest/500M_Resolution/PrecipVIIRSGeoO

  0%|          | 0/17 [00:00<?, ?it/s]

File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...


In [ ]:
os.path.dirname(f"{HOME}/Test/{file}")

In [5]:
HOME

'/uufs/chpc.utah.edu/common/home/civil-group1/Johnson'

In [3]:
#Upload nearest-snotel dictionaries
regionlist = ['Northwest', 'Northwest'] #'SouthernRockies', 'Southwest' ]
output_res = 500

#Get data
for region in regionlist:
    print(region)

Northwest
Northwest
